In [1]:
# Loading in HTML/CSS packages for use within the notebook
from IPython.core.display import HTML
HTML("<link href='https://fonts.googleapis.com/css?family=Passion+One' rel='stylesheet' type='text/css'><style>div.attn { font-family: 'Helvetica Neue'; font-size: 30px; line-height: 40px; color: #FFFFFF; text-align: center; margin: 30px 0; border-width: 10px 0; border-style: solid; border-color: #5AAAAA; padding: 30px 0; background-color: #DDDDFF; }hr { border: 0; background-color: #ffffff; border-top: 1px solid black; }hr.major { border-top: 10px solid #5AAA5A; }hr.minor { border: none; background-color: #ffffff; border-top: 5px dotted #CC3333; }div.bubble { width: 65%; padding: 20px; background: #DDDDDD; border-radius: 15px; margin: 0 auto; font-style: italic; color: #f00; }em { color: #AAA; }div.c1{visibility:hidden;margin:0;height:0;}div.note{color:red;}</style>")

# Lab Two: Classification
### Group A: Mai Loan Tran, Lei Jiang, Zachary Gill, & Samantha Sprague

## Table of Contents for Lab 2
* <a href="#Introduction">Introduction</a>
* <a href="#CreateModels">Data Preperation</a>
* <a href="#ModelAdvantages">Modeling and Evaluation</a>
* <a href="#InterpretFeatureImportance">Deployment</a>
* <a href="#Exceptional Work">Exceptional Work</a>
* <a href="#References">References</a>



<a id="Introduction"></a>
## Introduction

<a id="Data Preperation"></a>
## Data Preperation

**Requirement**:

• Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for
the analysis.

**Requirement**:

• Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created). 

<a id="Modeling and Evaluation"></a>
## Modeling and Evaluation

**Requirement**:

• Choose and explain your evaluation metrics that you will use (i.e., accuracy, precision, recall, F-measure, or any metric we have discussed). Why are the measure(s) appropriate for analyzing the results of your modeling? Give a detailed explanation
backing up any assertions.

**Answer**:

**Requirement**:
    
• Choose the method you will use for dividing your data into training and
testing splits (i.e., are you using Stratified 10-fold cross validation? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate.

**Answer**:

**Requirement**:

• Create three different classification/regression models (e.g., random forest, KNN, and SVM). Two modeling techniques must be new (but the third could be SVM or logistic regression). Adjust parameters as appropriate to increase generalization performance using your chosen metric.

**Answer**:

**Requirement**:

• Analyze the results using your chosen method of evaluation. Use
visualizations of the results to bolster the analysis. Explain any visuals and analyze why they are interesting to someone that might use this model.

**Answer**:

**Requirement**:

• Discuss the advantages of each model for each classification task, if any. If there are not advantages, explain why. Is any model better than another? Is the difference significant with 95% confidence? Use proper statistical comparison methods.

**Answer**:

**Requirement**:

• Which attributes from your analysis are most important? Use proper methods discussed in class to evaluate the importance of different attributes. Discuss the results and hypothesize about why certain attributes are more important than others
for a given classification task.

**Answer**:

<a id="Deployment"></a>
## Deployment

**Requirement**:
    
• How useful is your model for interested parties (i.e., the companies or
organizations that might want to use it for prediction)? How would you measure the model's value if it was used by these parties? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.? 

**Answer**:

<a id="Exceptional Work"></a>
## Exceptional Work

**Requirements**: 

• You have free reign to provide additional modeling. One idea: grid search parameters in a parallelized fashion and visualize the performances across attributes. Which parameters are most significant for making a good model for each classification algorithm?

<a id="References"></a>
## References